# ST590 - Project 2

Authored by Group 13 - Dave Bergeron, Xavier Genelin, and Maksim Nikiforov

## Part 1

Placeholder cell for work in progress

Relevant links:
- [The Netflix Tech Blog (browse for relevant posts)](https://netflixtechblog.com/)
- [Evolution of the Netflix Data Pipeline](https://netflixtechblog.com/evolution-of-the-netflix-data-pipeline-da246ca36905)
- [Rapid Event Notification System at Netflix](https://netflixtechblog.com/rapid-event-notification-system-at-netflix-6deb1d2b57d1)
- [Notebooks at Netflix: From analytics to engineering (sponsored by Netflix) - Michelle Ufford (Netflix) Kyle Kelley (Netflix)](https://learning.oreilly.com/videos/jupytercon-new-york/9781492025818/9781492025818-video322538/)
- [Petabytes Scale Analytics Infrastructure @Netflix](https://www.infoq.com/presentations/netflix-big-data-infrastructure/)


## Part 2

## Part 3